In [2]:
import os, torch
import pandas as pd
import numpy as np
from transformers import DistilBertForSequenceClassification, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

os.getcwd()

c:\Users\T-Gamer\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\T-Gamer\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\trainer_pt_utils.py:195: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:68.)
  device: Optional[torch.device] = torch.device("cuda"),


'c:\\Users\\T-Gamer\\Documents\\SideDrive\\UFMA\\2022.1\\Topicos Especiais (NLP)\\Exercicios\\Trabalho Final\\Implementação\\source'

In [3]:
def levenshtein(source:str, target:str) -> int :
    n = len(source)
    m = len(target)
    
    D = np.zeros((n + 1, m + 1), dtype=int)
    for i in range(1, n + 1) :
        D[i][0] = D[i - 1][0] + 1
    for j in range(1, m + 1) :
        D[0][j] = D[0][j - 1] + 1

    subst_cost = lambda x, y : 0 if x == y else 4
    for i in range(1, n + 1) :
        for j in range(1, m + 1) :
            D[i][j] = min([D[i - 1][j    ] + 1,
                           D[i - 1][j - 1] + subst_cost(source[i - 1], target[j - 1]),
                           D[i    ][j - 1] + 1])
    return D[n][m]
def distance(source:str, target:str) -> float :
    lev = levenshtein(source, target)
    if source in target or target in source : lev = lev - .5 
    return lev

In [4]:
class Dataset(torch.utils.data.Dataset) :
    def __init__(self, 
                 dataframe:pd.DataFrame, 
                 vocabulario:list[str],
                 max_length:int=512,
                 text_column:str='text',
                 label_column:str='class',
                 n_classes:int=1) :
        self.dataframe = dataframe
        self.vocabulario = vocabulario
        self.max_length = max_length
        self.text_column = text_column
        self.label_column = label_column
        self.n_classes = n_classes
    def __len__(self) :
        return self.dataframe.shape[0]
    def text_proxessing(self, text:str) :
        tokenlist = text.split()
        
        sequence = []
        for token in tokenlist :
            # if not token in self.vocabulario :
            #     voc_sort = self.vocabulario.copy()
            #     voc_sort.sort(key=lambda x : distance(token, x))
            #     token = voc_sort[0]
            sequence.append(self.vocabulario.index(token) + 1)

        mask = [1 for _ in range(len(sequence))]
        if len(sequence) > self.max_length :
            sequence = sequence[ : self.max_length]
            mask = mask[ : self.max_length]
        else :
            padding = [0 for _ in range(self.max_length - len(sequence))]
            sequence = sequence + padding
            mask = mask + padding
        
        return sequence, mask

    def __getitem__(self, index) :
        # print(index, type(index))
        sequence, mask = self.text_proxessing(self.dataframe.iloc[index][self.text_column])
        label = self.dataframe.iloc[index][self.label_column]
        if self.n_classes > 1 :
            label = [int(label)]
            # label = [([1] if i == int(label) else [0]) for i in range(self.n_classes)]
        else :
            label = [label]
        label = torch.tensor(label)
        return {'input_ids'      : torch.tensor(sequence),
                'attention_mask' : torch.tensor(mask),
                'labels'         : label}
        

In [6]:
def train(dataset:Dataset, embeddings:pd.DataFrame, epochs:int, output_dir:str, logging_dir:str) -> DistilBertForSequenceClassification :
    global resume_from_checkpoint
    model = DistilBertForSequenceClassification.from_pretrained(
        'distilbert-base-uncased',
        num_labels=dataset.n_classes
    )
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    embeddings_np_array = embeddings.values
    embeddings_np_array = np.append(np.zeros((1, embeddings_np_array.shape[1])), embeddings_np_array, axis=0)
    embeddings_module = torch.nn.Embedding.from_pretrained(
        embeddings=torch.tensor(embeddings_np_array),
        padding_idx=0
    )
    model.set_input_embeddings(embeddings_module)
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=epochs,
        per_device_train_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir=logging_dir,
        logging_steps=10
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset
    )
    trainer.train()
    return model

In [5]:
# Para dataset simples, sem folding
# Args
dataset_folder = "../resources/datasets/StanfordSentimentTreebank"
dataset_name   = "SST2Processed-train"

# Exec
embeddings_df = pd.read_csv(f"{dataset_folder}/embeddings/{dataset_name}.csv", index_col=0)
dataset_df    = pd.read_csv(f"{dataset_folder}/split/{dataset_name}.csv",      index_col=0)
output_dir    = f"../resources/output/{dataset_name}"
logging_dir   = f"../resources/logs/{dataset_name}"

dataset = Dataset(dataset_df, embeddings_df.index.tolist())
model = train(dataset, embeddings_df, epochs=5, output_dir=output_dir, logging_dir=logging_dir)
model.save_pretrained(output_dir + "/final")

In [7]:
# Para dataset com k-folding
# Args
k = 10
dataset_folder = "../resources/datasets/TwitterAirlines"
dataset_name   = "TweetsProcessed"
n_classes = 3
n_epochs = 3
label_column = "class"
# Exec
folds = []
for i_fold in range(k) :
    dataset_fold_name = f"{dataset_name}_Fold{i_fold + 1}"
    fold_df = pd.read_csv(f"{dataset_folder}/folds/{dataset_fold_name}.csv", index_col = 0)
    folds.append(fold_df)
for i_fold in range(1, k) :
    dataset_fold_name = f"{dataset_name}_Fold{i_fold + 1}"
    embeddings_df = pd.read_csv(f"{dataset_folder}/embeddings/{dataset_fold_name}.csv", index_col=0)
    dataset_df    = pd.concat(folds[ : i_fold] + folds[i_fold + 1 : ])
    
    output_dir    = f"../resources/output/{dataset_fold_name}"
    logging_dir   = f"../resources/logs/{dataset_fold_name}"
    for dir in (output_dir, logging_dir) :
        if not os.path.exists(dir) : 
            os.makedirs(dir)

    dataset = Dataset(dataset_df, embeddings_df.index.tolist(), label_column=label_column, n_classes=n_classes)
    model = train(dataset, embeddings_df, epochs=n_epochs, output_dir=output_dir, logging_dir=logging_dir)
    model.save_pretrained(output_dir + "/final")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classi

{'loss': 1.1321, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}


  1%|          | 20/2472 [09:16<18:55:01, 27.77s/it]

{'loss': 1.1136, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.02}


  1%|          | 30/2472 [13:47<18:23:53, 27.12s/it]

...

{'loss': 0.7054, 'learning_rate': 4.2393509127789046e-05, 'epoch': 0.97}


 33%|███▎      | 810/2472 [6:05:04<12:18:30, 26.66s/it]

{'loss': 0.7028, 'learning_rate': 4.213995943204868e-05, 'epoch': 0.98}


 33%|███▎      | 812/2472 [6:05:58<12:23:25, 26.87s/it]

ValueError: 'nan' is not in list